In [425]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [426]:
train=pd.read_csv("/content/train_data.csv")
test=pd.read_csv("/content/test_data.csv")

In [427]:
subtask=[0,0,0,0]

In [428]:
subtask[0]=train.shape[0]

In [429]:
S,cnt=0,0
for i in range(train.shape[0]):
    if (train["Credit_Utilization_Ratio"][i]>=25):
      S+=train["Monthly_Inhand_Salary"][i]
      cnt+=1
subtask[1]=(int)(S/cnt)

In [430]:
Months={}
for month in train["Month"]:
  if month in Months:
      Months[month]+=1
  else:
      Months[month]=1
for i in Months:
    if i==1:
      subtask[2]+=1

In [431]:
for i in train["SSN"]:
  if i[-1]=='0' and i[-2]=='2':
      subtask[3]+=1

In [432]:
"""numericFeatures=["Age","Annual_Income","Monthly_Inhand_Salary"
,"Num_Bank_Accounts","Num_Credit_Card","Interest_Rate",
"Num_of_Loan","Delay_from_due_date","Num_of_Delayed_Payment",
"Changed_Credit_Limit","Num_Credit_Inquiries",
"Outstanding_Debt","Credit_Utilization_Ratio",
"Amount_invested_monthly","Monthly_Balance"]
stringFeatures=["Occupation","Payment_Behaviour","Payment_of_Min_Amount"]
features=numericFeatures+stringFeatures"""

In [433]:
"""for i in numericFeatures:
    for j in train[i]:
        if (isinstance(j, str) and j[-1]=='_'):
            print (i)
            break
#verificam datele numerice care au _ la final
def errors_(x):
    if isinstance(x, str):
        x = x.strip("_")  # strip underscores from the ends
    if (x==''):
      return 0
    return float(x)
    """

Age
Annual_Income
Num_of_Loan
Num_of_Delayed_Payment
Changed_Credit_Limit
Outstanding_Debt
Amount_invested_monthly
Monthly_Balance


In [434]:
"""errors = [
    "Age",
    "Annual_Income",
    "Num_of_Loan",
    "Num_of_Delayed_Payment",
    "Changed_Credit_Limit",
    "Outstanding_Debt",
    "Amount_invested_monthly",
    "Monthly_Balance"
]
train[errors]=train[errors].applymap(errors_)
test[errors]=test[errors].applymap(errors_)"""

/tmp/ipython-input-3457285074.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train[errors]=train[errors].applymap(errors_)
/tmp/ipython-input-3457285074.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test[errors]=test[errors].applymap(errors_)


In [435]:
"""features_correct = [col for col in features if train[col].shape[0] == train.shape[0]]"""

In [441]:
"""x_train=train[features_correct]
y_train=train["Credit_Score"]
x_test=train[features_correct]"""

In [437]:
"""
preprocessor=ColumnTransformer(transformers=[
    ('numeric',StandardScaler(),numericFeatures),
    ('string',OneHotEncoder(handle_unknown='ignore'),stringFeatures),
])
"""

In [438]:
"""
model=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('regressor',RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        n_jobs=-1,
        random_state=42
    ))
])
"""

In [439]:
"""
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
"""

In [442]:
numeric_features=["Age","Annual_Income","Monthly_Inhand_Salary","Num_Bank_Accounts","Interest_Rate","Num_of_Loan",
                  "Delay_from_due_date","Num_of_Delayed_Payment","Changed_Credit_Limit","Num_Credit_Inquiries",
                  "Credit_Utilization_Ratio","Total_EMI_per_month","Amount_invested_monthly",
                  "Monthly_Balance"]
# Clean train set
train[numeric_features] = (
    train[numeric_features]
    .astype(str)
    .applymap(lambda x: x.replace("_", ""))  # remove underscores
    .replace("", np.nan)                     # convert empty strings to NaN
    .apply(pd.to_numeric, errors="coerce")   # safely convert to float
)

# Clean test set
test[numeric_features] = (
    test[numeric_features]
    .astype(str)
    .applymap(lambda x: x.replace("_", ""))
    .replace("", np.nan)
    .apply(pd.to_numeric, errors="coerce")
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
preprocessor=ColumnTransformer(transformers=[
    ("numeric",StandardScaler(),numeric_features)
])
features=numeric_features
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
model=Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("regressor",RandomForestClassifier(n_estimators=100,random_state=5))
])
model.fit(train[features],train["Credit_Score"])
y_pred=model.predict(test[features])

/tmp/ipython-input-791995153.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: x.replace("_", ""))  # remove underscores
/tmp/ipython-input-791995153.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: x.replace("_", ""))


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [443]:
solution=pd.DataFrame({
    'subtaskID':[1,2,3,4],
    'datapointID':[1]*4,
    'answer':subtask
})
for i in range(test.shape[0]):
    id=test['ID'][i]
    sol=pd.DataFrame({
        'subtaskID':[5],
        'datapointID':[id],
        'answer':y_pred[i]
    })
    solution=pd.concat([solution,sol],ignore_index=True)
solution.to_csv("submission.csv",index=False)